# sample_local.ipynb

This notebook is a short example of how the MagicHour pipeline works. The code is an adaptation of driver.py located in api/local/sample/. Each section in this notebook describes a separate step in the pipeline that we use to process log files. The functions that are used here are intermediate driver functions that use the underlying MagicHour API. (The driver functions provide logging and measurements of function execution time.)

In [1]:
import os
import time

from magichour.api.local.sample.steps.evalapply import evalapply_step
from magichour.api.local.sample.steps.event import event_step
from magichour.api.local.sample.steps.genapply import genapply_step
from magichour.api.local.sample.steps.genwindow import genwindow_step
from magichour.api.local.sample.steps.preprocess import preprocess_step
from magichour.api.local.sample.steps.template import template_step
from magichour.api.local.sample.driver import *

from magichour.api.local.util.log import get_logger, log_time
from magichour.api.local.util.pickl import read_pickle_file, write_pickle_file

magichour_root = os.path.dirname(os.getcwd())
data_dir = os.path.join(magichour_root, "magichour", "api", "local", "sample", "data")

In [2]:
run = 0

runList = ['.500k','.1M','.2M','.4M']
support = ['2','5','9','19']

data_dir = os.path.join("/","Users","dgrossman","data","testing")
loglines_file = os.path.join(data_dir,'output','%s%s'%('outputPreProcess',runList[run]))
gen_templates_file = os.path.join(data_dir,'output','%s%s'%('outputTemplates',runList[run]))
eval_loglines_file = os.path.join(data_dir,'output','%s%s'%('outputEvalLogLines',runList[run]))
gen_windows_file = os.path.join(data_dir,'output','%s%s'%('outputGenWindow',runList[run]))
gen_events_file = os.path.join(data_dir,'output','%s%s'%('outputGenEvents' ,runList[run]))
timed_events_file = os.path.join(data_dir,'output','%s%s'%('outputTimedEvents' ,runList[run]))


In [3]:
log_file = os.path.join(data_dir, "logs", '%s%s'%("tbird.log",runList[run]))
transforms_file = os.path.join(data_dir, "sample.transforms")

read_lines_args = [{}, 0, 10]
read_lines_kwargs = {"skip_num_chars": 22}
logcluster_kwargs = {"support": "%s"%(support[run])}
paris_kwargs = {"r_slack": 0.0, "num_iterations": 3}
gen_window_kwargs = {"window_size": 60, "tfidf_threshold": 0.0}

# only return 10000 itemsets...iterations = -1 will return all
fp_growth_kwargs = {"min_support": 0.005, "iterations": 10000, "tfidf_threshold": 1.0}



---

# Preprocess

The preprocess step takes a log file and transforms it into an iterable of LogLine named tuples. While it is possible to do this without altering the original line via the get_lines() function in templates/templates.py, it is suggested that you write some transforms (see code below) and use the get_transformed_lines() function in order to perform normalizations like converting instances of things like machine/user names or IP addresses to standard tokens. Doing this will produce much better results in the Template step. We maintain the replacements in the LogLine named tuple in order to provide the ability to reconstruct LogLines throughout the pipeline.

Calculate transformed loglines from a log file+transform file.

In [4]:
start = time.time()

loglines = preprocess_step(log_file, transforms_file, *read_lines_args, **read_lines_kwargs)
write_pickle_file(loglines, loglines_file)

2016-03-01 15:21:44,546 [INFO] [magichour.api.local.sample.steps.preprocess] Reading log lines from file: /Users/dgrossman/data/testing/logs/tbird.log.500k
2016-03-01 15:21:44,547 [INFO] [magichour.api.local.sample.steps.preprocess] Reading transforms from file: /Users/dgrossman/data/testing/sample.transforms
2016-03-01 15:21:44,551 [INFO] [magichour.api.local.sample.steps.preprocess] Transforming log lines...
2016-03-01 15:22:23,303 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputPreProcess.500k


Read transformed loglines from a prepared pickle file.

In [5]:
#loglines = read_pickle_file(loglines_file)

You can examine and interact with the loglines output in the cell below.

In [6]:
#loglines[0]

---

# Template

The template step takes in an iterable of LogLine named tuples and produces an iterable of Template named tuples. Ideally, this is the output of the previous preprocess step, however, these functions can be run independently as long as you marshal your data into iterable LogLines. In fact, the MagicHour API was designed with this in mind since we anticipate that users will want to mix and match different pipeline modules.

We provide two possible templating algorithms: LogCluster and StringMatch. Additional details about LogCluster is  available at http://ristov.github.io/logcluster/. For additional details about StringMatch, see the paper "[One Graph Is Worth a Thousand Logs: Uncovering Hidden Structures in Massive System Event Logs](http://link.springer.com/chapter/10.1007%2F978-3-642-04180-8_32)" by Aharon, Barash, Cohen, and Mordechai. There is also a [video available](http://videolectures.net/ecmlpkdd09_barash_gwtluhsmsel/) that provides more information about StringMatch.

*Note: The name "StringMatch" was taken from another [paper](http://users.cis.fiu.edu/~taoli/pub/liang-cikm2011.pdf) (Aharon et al do not name their algorithm).*

### LogCluster

Generate templates using the LogCluster algorithm.

In [7]:
gen_templates = template_step(loglines, "logcluster", **logcluster_kwargs)
write_pickle_file(gen_templates, gen_templates_file)

templateGen_time = time.time() - start

2016-03-01 15:22:38,012 [INFO] [magichour.api.local.sample.steps.template] Running logcluster... ({'support': '2'})
2016-03-01 15:22:46,475 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputTemplates.500k


### **(\*\*WIP\*\*)** StringMatch

Generate tempaltes using the StringMatch algorithm. StringMatch uses cosine similarity to group log lines which are alike. You should use LogCluster if you aren't tolerant of lossy templating -- though it should be noted that the preprocess step should help to mitigate the loss from StringMatch.

In [8]:
#gen_templates = template_step(loglines, "stringmatch")
#write_pickle_file(gen_templates, gen_templates_file)

Read templates from a prepared pickle file.

In [9]:
#gen_templates = read_pickle_file(gen_templates_file)

You can examine and interact with the templates output in the cell below.

In [10]:
gen_templates[75]
print len (gen_templates)

1564


---

# Apply Templates

The apply templates step takes in an iterable of LogLine named tuples and an iterable of Template named tuples (i.e. output of previous templating step). In this instance, we are applying the generated templates on the same log file that they came from. We believe that both this step and the Apply Events step (further down) are the only two steps that are needed to be implemented in a streaming fashion (to be able to keep up with log file ingest rates). The remainder of the steps described in this notebook could theoretically be run offline nightly (i.e. batch processing).

The output of the apply templates step is an iterable of TimedTemplate named tuples, representing the instances of each template that were found in the log file. If the template_id is -1 in a TimedTemplate, then that means that no template was found that matches that particular line.

Create timed templates by applying templates generated from the last step (Template) over an iterable of LogLines.

In [11]:
start = time.time()
eval_loglines = genapply_step(loglines, gen_templates)
write_pickle_file(eval_loglines, eval_loglines_file)

2016-03-01 15:22:46,526 [INFO] [magichour.api.local.sample.steps.genapply] Applying templates to lines...
2016-03-01 15:23:09,879 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputEvalLogLines.500k


Read timed templates from a prepared pickle file.

In [12]:
#eval_loglines = read_pickle_file(eval_loglines_file)

templateEval_time = time.time() - start

You can examine and interact with the timed_templates output in the cell below.

In [13]:
#eval_loglines[0]

---

# Window

The window step takes in an iterable of TimedTemplate named tuples (i.e. the output of the apply templates step) and returns an iterable of sets containing TimedTemplate instances. Each of these windows represent a time range in which the contained template_id's co-occurred. . In effect, we are creating transactions by grouping all TimedTemplates within *window_size*. These transactions will be passed to the next step which will perform market basket analysis on them in order to identify frequently co-occurring itemsets.

Create windows using timed templates generated from the last step (Apply Templates).

In [14]:
start = time.time()
gen_windows = genwindow_step(eval_loglines, **gen_window_kwargs)
write_pickle_file(gen_windows, gen_windows_file)

2016-03-01 15:23:28,423 [INFO] [magichour.api.local.sample.steps.genwindow] Creating model gen windows from timed_templates...
2016-03-01 15:23:29,204 [INFO] [magichour.api.local.sample.steps.genwindow] ==========Custom post processing for sample data==========
2016-03-01 15:23:29,205 [INFO] [magichour.api.local.sample.steps.genwindow] Removing junk drawer entries from each window's template_ids. (template_id = -1)
2016-03-01 15:23:29,307 [INFO] [magichour.api.local.sample.steps.genwindow] Removing duplicate entries from each window's template_ids.
2016-03-01 15:23:29,383 [INFO] [magichour.api.local.sample.steps.genwindow] Applying a tfidf filter to each window's template_ids. (threshold = 0.0)
2016-03-01 15:23:29,440 [INFO] [magichour.api.local.sample.steps.genwindow] ==========End custom post processing==========
2016-03-01 15:23:29,441 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputGenWindow.500k


Read windows from a prepared pickle file.

In [15]:
gen_windows = read_pickle_file(gen_windows_file)


2016-03-01 15:23:29,462 [INFO] [magichour.api.local.util.pickl] Reading pickle file: /Users/dgrossman/data/testing/output/outputGenWindow.500k


You can examine and interact with the window output in the cell below.

In [16]:
gen_windows

[[896],
 [449],
 [449],
 [449],
 [449],
 [449],
 [896],
 [449],
 [449],
 [449],
 [449],
 [896],
 [449],
 [449],
 [896, 449],
 [449],
 [449],
 [449],
 [449],
 [896],
 [449],
 [449],
 [449],
 [449],
 [449],
 [449],
 [896],
 [449],
 [449],
 [449],
 [896],
 [449],
 [896, 449],
 [449],
 [896],
 [449],
 [449],
 [449],
 [896],
 [449],
 [449],
 [449],
 [896],
 [449],
 [896],
 [449],
 [896, 449],
 [896],
 [449],
 [449],
 [896],
 [449],
 [449],
 [896],
 [449],
 [449],
 [896],
 [896],
 [1281,
  514,
  260,
  647,
  539,
  929,
  680,
  1068,
  1217,
  584,
  207,
  1361,
  994,
  483,
  100,
  357,
  1393,
  244,
  638],
 [514,
  1544,
  649,
  1546,
  23,
  1550,
  783,
  21,
  1113,
  1305,
  24,
  665,
  1306,
  27,
  28,
  29,
  30,
  31,
  32,
  673,
  802,
  1188,
  805,
  730,
  732,
  682,
  427,
  1452,
  240,
  686,
  815,
  115,
  948,
  1461,
  184,
  692,
  58,
  70,
  831,
  321,
  11,
  710,
  1479,
  15,
  1228,
  1483,
  1484,
  1487,
  1363,
  1444,
  90,
  1499,
  860,
  858,
 

In [17]:
len(gen_windows)

1830

---

# Event

### fp_growth

Generate events by applying the fp_growth algorithm on windows created from last step (Window).

In [18]:
#gen_events = event_step(gen_windows, "fp_growth", **fp_growth_kwargs)
#write_pickle_file(gen_events, gen_events_file)

### PARIS

Generate events by applying the PARIS algorithm on windows created from last step (Window).

In [19]:

gen_events = event_step(gen_windows, "paris", **paris_kwargs)
write_pickle_file(gen_events, gen_events_file)
eventGen_time = time.time() - start

2016-03-01 15:23:29,972 [INFO] [magichour.api.local.sample.steps.event] Running PARIS algorithm... ({'r_slack': 0.0, 'num_iterations': 3})
2016-03-01 15:24:37,632 [INFO] [magichour.api.local.sample.steps.event] ==========Custom post processing for sample data==========
2016-03-01 15:24:37,633 [INFO] [magichour.api.local.sample.steps.event] Skipping tfidf filter
2016-03-01 15:24:37,633 [INFO] [magichour.api.local.sample.steps.event] ==========End custom post processing==========
2016-03-01 15:24:37,634 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputGenEvents.500k


Read events from a prepared pickle file.

In [20]:
gen_events = read_pickle_file(gen_events_file)

2016-03-01 15:24:37,640 [INFO] [magichour.api.local.util.pickl] Reading pickle file: /Users/dgrossman/data/testing/output/outputGenEvents.500k


You can examine and interact with the event output in the cell below.

gen_events

Below are the templates which comprise each event that we identified in this step. In this example we've found what looks to be:

* a DHCP request
* some type of change in the network
* an SSH login

The parameters that we chose, along with the TF-IDF filtering done in the windowing and event discovery steps, made the discovery pretty restrictive. Playing around with the settings and removing the filtering will cause the pipeline to discover other event types.

template_d = {template_id : template for (template_id, template) in [(template.id, template) for template in gen_templates]}
e = []
for event in gen_events:
    ts = []
    for template_id in event.template_ids:
        ts.append("%s: %s" % (template_id, template_d[template_id].raw_str))
    e.append(ts)
from pprint import pprint
pprint(e)

---

# Apply Events

Create timed templates by applying templates generated from the last step (Template) over an iterable of LogLines.

The way this currently works is by determining which template for each event is the least frequently occurring template in timed_templates. By using the least frequently occuring template, we are guaranteed to not discover more than the maximum number of events allowed in a given list of timed_templates. 

For each event's least frequently occurring template, we construct a "window" around each instance in timed_templates, where a "window" is a collection of all of the timed_templates that occurred within the specified window size (60 secs by default). Within each window, for each of the remaining template types belonging to the event in question, we say that the logline with the highest Jaccard similarity score between its replacement values and the original frequently occuring template's replacement values. The logic here is that the closer the Jaccard similarity score, the more likely that the two loglines are talking about the same (machine, IP address, etc.).

In [21]:
start = time.time()
timed_events = evalapply_step(gen_events, eval_loglines)

write_pickle_file(timed_events, timed_events_file)
eventEval_time = time.time() - start

2016-03-01 15:24:37,650 [INFO] [magichour.api.local.sample.steps.evalapply] Applying events to timed templates...
2016-03-01 15:24:37,861 [INFO] [magichour.api.local.modeleval.apply] Using single-thread to apply events
2016-03-01 15:24:37,861 [INFO] [magichour.api.local.modeleval.apply] Processing event 0 of 16
2016-03-01 15:25:18,834 [INFO] [magichour.api.local.modeleval.apply] Processing event 10 of 16
2016-03-01 15:25:18,922 [INFO] [magichour.api.local.util.pickl] Writing data to pickle file: /Users/dgrossman/data/testing/output/outputTimedEvents.500k


Read timed events from a prepared pickle file.

In [22]:
timed_events = read_pickle_file(timed_events_file)

2016-03-01 15:25:19,695 [INFO] [magichour.api.local.util.pickl] Reading pickle file: /Users/dgrossman/data/testing/output/outputTimedEvents.500k


You can examine and interact with the window output in the cell below.

In [23]:
timed_events

[TimedEvent(event_id='05a02862-d724-4e5c-80bf-ffba5123447c', timed_templates=[DistributedLogLine(ts=1131566268.0, text='#1# logger: Kickstart Install: Prepare speconf_sync to work', processed='#INT# logger: Kickstart Install: Prepare speconf_sync to work', proc_dict={'INT': ['1']}, template='\\#INT\\#\\ logger\\:\\ Kickstart\\ Install\\:\\ Prepare\\ speconf\\_sync\\ to\\ work$', templateId=680, template_dict=defaultdict(<type 'list'>, {})), DistributedLogLine(ts=1131566268.0, text='#1# logger: Kickstart Install: Torque Mom and Maui speconf Tree', processed='#INT# logger: Kickstart Install: Torque Mom and Maui speconf Tree', proc_dict={'INT': ['1']}, template='\\#INT\\#\\ logger\\:\\ Kickstart\\ Install\\:\\ Torque\\ Mom\\ and\\ Maui\\ speconf\\ Tree$', templateId=584, template_dict=defaultdict(<type 'list'>, {})), DistributedLogLine(ts=1131566268.0, text='#1# logger: Kickstart: setup firstboot', processed='#INT# logger: Kickstart: setup firstboot', proc_dict={'INT': ['1']}, template='\

In [24]:
len(timed_events)

7048

In [25]:
[te.event_id for te in timed_events]

['05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-4e5c-80bf-ffba5123447c',
 '05a02862-d724-

---

In [26]:
print templateGen_time,templateEval_time, eventGen_time, eventEval_time


61.9576120377 41.8886129856 69.2128119469 42.0422689915
